In [76]:
!pip install praw
!pip install vaderSentiment

In [77]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import re
import praw
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [78]:
# Helper function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text.strip().lower()

In [79]:
# Sentiment Analysis using VADER and TextBlob
def analyze_sentiment(text):
    cleaned_text = clean_text(text)

    # VADER Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()
    vader_score = analyzer.polarity_scores(cleaned_text)['compound']

    textblob_score = TextBlob(cleaned_text).sentiment.polarity
    
    combined_score = (vader_score + textblob_score) / 2

    # Normalize to 1-100 scale
    sentiment_score = int((combined_score + 1) * 50)
    return sentiment_score

In [80]:
# Function to classify sentiment
def classify_sentiment(score):
    if score <= 40:
        return "Strong Sell"
    elif 41 <= score < 60:
        return "Neutral"
    elif score >= 60:
        return "Strong Buy"

In [81]:
#Funtion to fetch from News API 
def fetch_news_sentiment(stock_symbol, num_articles=5):
    api_key = user_secrets.get_secret("NEWS_API_KEY")
    url = f'https://newsapi.org/v2/everything?q={stock_symbol}&language=en&apiKey={api_key}'
    scores = []

    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        articles = data.get('articles', [])
        
        print(f"\nAnalyzing {num_articles} News Articles for {stock_symbol} (sorted by newest)...\n")
        
        for article in articles[:num_articles]:
            title = article.get('title', '')
            description = article.get('description', '')

            # Ensure valid title and description
            if title and description and "[Removed]" not in title and "[Removed]" not in description:
                text = title + " " + description
                sentiment_score = analyze_sentiment(text)
                scores.append(sentiment_score)
                print(f"Title: {title}\nPublished At: {article.get('publishedAt', '')}\nSentiment Score: {sentiment_score}\n")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
    
    return scores

In [82]:
# Fetch Reddit Posts (Reddit API using PRAW)
def fetch_reddit_sentiment(stock_symbol, num_posts=5):
    reddit = praw.Reddit(
            client_id = user_secrets.get_secret("REDDIT_CLIENT_ID"),
            client_secret = user_secrets.get_secret("REDDIT_CLIENT_SECRET"),
            user_agent = user_secrets.get_secret("REDDIT_USER_AGENT"),
            username = user_secrets.get_secret("REDDIT_USER_NAME"),
            password = user_secrets.get_secret("REDDIT_PASSWORD"), 
            )
    scores = []
    
    subreddit = reddit.subreddit('all')
    #posts = subreddit.search(stock_symbol, limit=num_posts, sort='new')
    posts = subreddit.search(stock_symbol, limit=num_posts)

    print(f"\nAnalyzing {num_posts} Reddit Posts for {stock_symbol}...\n")
    
    for post in posts:
        text = post.title + " " + post.selftext
        sentiment_score = analyze_sentiment(text)
        scores.append(sentiment_score)
        print(f"Title: {post.title}\nSentiment Score: {sentiment_score}\n")
    
    return scores

In [83]:
# Main function to fetch and aggregate sentiment
def fetch_and_analyze_stock_sentiment(stock_symbol, num_posts=10):
    # Fetch sentiments from different platforms
    news_scores = fetch_news_sentiment(stock_symbol, num_articles=num_posts)
    reddit_scores = fetch_reddit_sentiment(stock_symbol, num_posts=num_posts)

    all_scores = news_scores + reddit_scores
    if all_scores:
        overall_score = sum(all_scores) / len(all_scores)
        overall_classification = classify_sentiment(overall_score)

        print(f"\nOverall Sentiment for {stock_symbol}: {overall_classification} ({overall_score:.2f})")
        if overall_classification == "Strong Buy": return 1
        if overall_classification == "Neutral": return 0
        if overall_classification == "Strong Sell": return -1 
        
    else:
        print(f"\nNo data available to analyze sentiment for {stock_symbol}.")

In [84]:
# Fetch News Articles for Stock Market (Global or Country-Specific)
def fetch_market_sentiment(market_type="global", country="US", num_articles=5):
    api_key = user_secrets.get_secret("NEWS_API_KEY")

    # Based on the market type, decide the query
    if market_type == "global":
        query = "stock market"
    elif market_type == "country":
        query = f"stock market {country}"
    else:
        print("Invalid market type. Please choose 'global' or 'country'.")
        return None  # Return None for invalid inputs

    url = f'https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&language=en&apiKey={api_key}'
    scores = []

    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        articles = data.get('articles', [])
        
        if market_type == "country":
            print(f"\nAnalyzing {num_articles} News Articles for {market_type} Stock Market ({country})\n")
        else:
            print(f"\nAnalyzing {num_articles} News Articles for {market_type} Stock Market\n")

        for article in articles[:num_articles]:
            title = article.get('title', '')
            description = article.get('description', '')
            published_at = article.get('publishedAt', '')

            if title and description and "[Removed]" not in title and "[Removed]" not in description:
                text = title + " " + description
                sentiment_score = analyze_sentiment(text)
                scores.append(sentiment_score)

                # Display article details (optional)
                print(f"Title: {title}")
                print(f"Description: {description}")
                print(f"Published At: {published_at}")
                print(f"Sentiment Score: {sentiment_score}")
                print("-" * 80)

        # Calculate the overall sentiment score if there are any scores
        if scores:
            overall_score = sum(scores) / len(scores)
            return overall_score
        else:
            print("No valid articles found for sentiment analysis.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return None

**Example usage**

In [85]:
fetch_and_analyze_stock_sentiment('Zomato', num_posts=10)


Analyzing 10 News Articles for Zomato (sorted by newest)...

Title: Billionaire food app CEO wants you to pay for the privilege of working with him
Published At: 2024-11-21T07:26:05Z
Sentiment Score: 64

Title: Zomato, RIL, and BSE among stocks that mutual funds bought and sold in November
Published At: 2024-12-12T06:32:15Z
Sentiment Score: 52

Title: Board of Paytm's Singapore arm approves stake sale in Japanese firm PayPay Corporation
Published At: 2024-12-06T12:09:42Z
Sentiment Score: 60

Title: Food app CEO seeks unpaid chief of staff willing to pay $20,000 for the role
Published At: 2024-11-21T11:15:00Z
Sentiment Score: 48

Title: India’s Zomato ordered to pay $94.8m in tax penalties
Published At: 2024-12-13T16:44:25Z
Sentiment Score: 67

Title: Swiggy and Zomato stocks drop due to Amazon’s new quick commerce venture in India
Published At: 2024-12-11T09:10:07Z
Sentiment Score: 59

Title: Swiggy tops expectations with best major listing in a decade
Published At: 2024-11-13T23:58:2

0

In [86]:
fetch_market_sentiment(market_type="country", country="India", num_articles=5)


Analyzing 5 News Articles for country Stock Market (India)

Title: Life Insurance Corp pares 2% stake in NMDC
Description: State-owned Life Insurance Corporation of India (LIC) has reduced its stake in NMDC Ltd by 2%, decreasing its holding from 7.6% to 5.6%.  LIC sold over 5.91 crore shares through market sales between September 2023 and December 2024.  This follows LIC's recent…
Published At: 2024-12-13T16:22:43Z
Sentiment Score: 57
--------------------------------------------------------------------------------
Title: Affirm Secures $4 Billion Loan Deal With Sixth Street to Boost BNPL Growth
Description: On Friday, fintech company Affirm Holdings, Inc (NASDAQ:AFRM) stock gained after the company announced a long-term capital partnership with Sixth Street to...
Published At: 2024-12-13T16:01:04Z
Sentiment Score: 71
--------------------------------------------------------------------------------
Title: India must do more to tax its super-rich, France's Piketty says
Description: Frenc

69.6

In [87]:
fetch_market_sentiment(market_type="global", num_articles=5)


Analyzing 5 News Articles for global Stock Market

Title: Broadcom, Nvidia, Tesla, RH, Boeing, Affirm, Upstart, Salesforce, and More Movers - Barron's
Description: Broadcom, Nvidia, Tesla, RH, Boeing, Affirm, Upstart, Salesforce, and More MoversBarron's Stock market today: Dow, S&P 500 head for losing week, Broadcom surges after earningsYahoo Finance Stocks making the biggest moves before the bell: Broadcom, RH, Tesla, …
Published At: 2024-12-13T16:52:42Z
Sentiment Score: 52
--------------------------------------------------------------------------------
Title: Donegal group director sells $202,310 in stock
Description: Donegal group director sells $202,310 in stock
Published At: 2024-12-13T16:50:57Z
Sentiment Score: 50
--------------------------------------------------------------------------------
Title: ROSEN, A LEADING LAW FIRM, Encourages Quanterix Corporation Investors to Inquire About Securities Class Action Investigation - QTRX
Description: ROSEN, A LEADING LAW FIRM, Encourage

47.4